In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px
import numpy as np
import hashlib as hl
import statistics as st

directory_data = "../data"
athletes = pd.read_csv(f"{directory_data}/athlete_events.csv")
regions = pd.read_csv(f"{directory_data}/noc_regions.csv")

#anonymisering 
athletes["Name"] = athletes["Name"].astype(str)
hashes = athletes["Name"].apply(lambda x: hl.sha256(x.encode()).hexdigest())
hashes.head()
athletes.insert(1, "Hash values", hashes)
athletes.head()
athletes.drop(columns=["Name"], inplace=True)

merged = pd.merge(athletes, regions, on="NOC", how="outer")



In [66]:
athlete_ages = pd.read_csv(f"{directory_data}/athlete_events_fixed_age.csv")

#anonymisera - borde göra en funktion för det
athlete_ages["Name"] = athlete_ages["Name"].astype(str)
hashes1 = athlete_ages['Name'].apply(lambda x: hl.sha256(x.encode()).hexdigest())
athlete_ages.insert(1, "Hash values", hashes1)
athlete_ages.drop(columns=["Name"],inplace=True)

### Australien - Hur har åldrarna på deltagarna sett ut genom åren?


In [67]:
australia_rows = athlete_ages[(athlete_ages['NOC'] == 'AUS') | (athlete_ages['NOC'] == 'ANZ')]
australia_rows_uniqueID = australia_rows.drop_duplicates(subset=['ID'])

fig_hist1 = px.histogram(
    australia_rows_uniqueID, 
    x='Age', 
    nbins=50,
    title="Ages of Australian olympians", 
)

fig_hist1.show()

### Vilka åldrar har vunnit flest medaljer för Australien?

så använda australia_rows



In [ ]:
australia_rows_medals = australia_rows[australia_rows['Medal'].notna()]
medals = australia_rows_medals.groupby(['Age', 'Medal']).size().unstack(fill_value=0) #count medals per age

medals["Total"] = medals['Gold'] + medals['Silver'] + medals['Bronze']
top10= medals.sort_values(by="Total", ascending=False).head(10).reset_index()

medal_values = ["Bronze", "Silver", "Gold"]
fig_bar = px.bar(top10, x=medal_values, y='Age', color_discrete_map={"Gold":"#FFD700", "Silver": "#C0C0C0", "Bronze": "#CD7F32"})

fig_bar.show()

In [68]:
australia_rows_uniqueID['Age'].describe()

count    3868.000000
mean       24.074199
std         5.255404
min        13.000000
25%        21.000000
50%        23.000000
75%        27.000000
max        62.000000
Name: Age, dtype: float64

In [69]:
australia_rows_uniqueID[australia_rows_uniqueID['Age']==62]

,ID,Hash values,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
195308,97913,f0f458f2a4a6d51f55a55c3f08aded09a65ffbb0b68687...,M,62.0,NaN,NaN,Australia,AUS,1932 Summer,1932,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN


# Sport: Rowing

### Hur har åldrarna inom rodd sett ut genom åren?

In [70]:
rowing_rows = merged[merged['Sport']=='Rowing']
rowing_rows_uniqueID = rowing_rows.drop_duplicates(subset=['ID'])

fig_hist2 = px.histogram(
    rowing_rows_uniqueID, 
    x='Age', 
    nbins=50, 
    title="Histogram for Ages in Rowing all countries"
)

fig_hist2.show()


I början är det ganska spritt. Från runt 50-talet håller sig medelåldern runt 25 år. 

In [71]:
rowing_rows_uniqueID['Age'].describe()

count    7195.000000
mean       24.041140
std         3.806587
min        11.000000
25%        21.000000
50%        24.000000
75%        26.000000
max        60.000000
Name: Age, dtype: float64

### FRÅGA: Vilka har fått flest medaljer i rodd (topp 10)?

In [ ]:
rowing_rows_medals = rowing_rows[rowing_rows['Medal'].notna()]
medals = rowing_rows_medals.groupby(['Team', 'Medal']).size().unstack(fill_value=0) #count medals per Team

medals["Total"] = medals['Gold'] + medals['Silver'] + medals['Bronze']
top10= medals.sort_values(by="Total", ascending=False).head(10).reset_index()

medal_values = ["Bronze", "Silver", "Gold"]
fig_bar = px.bar(top10, x=medal_values, y='Team', color_discrete_map={"Gold":"#FFD700", "Silver": "#C0C0C0", "Bronze": "#CD7F32"}, title="Olympic medals in rowing per 10 top team")

fig_bar.show()


### När var rodd med i OS för första gången?

In [73]:
np.sort(rowing_rows['Year'].unique())

array([1900., 1904., 1906., 1908., 1912., 1920., 1924., 1928., 1932.,
       1936., 1948., 1952., 1956., 1960., 1964., 1968., 1972., 1976.,
       1980., 1984., 1988., 1992., 1996., 2000., 2004., 2008., 2012.,
       2016.])

### Under hur lång period tävlar de i rodd?
hur räknar jag ut vilka år en enda person varit med?
och gör det för varje hash value?
och sen gör ett genomsnitt hur många spel de varit med i?

får kanske gör en foorlop som går igenom varje unikt hash värde
och kollar hur många år de var med och lägger i en lista utanför loopen
när allas antal år är i listan så kör jag medel år på listan 

In [74]:
rowing_rows

,ID,Hash values,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
293,1468.0,50d687f37c20d156e7320001e7141b3dbc247d92dd54c2...,M,23.0,185.0,73.0,Algeria,ALG,2004 Summer,2004.0,Summer,Athina,Rowing,Rowing Men's Single Sculls,NaN,Algeria,NaN
300,1518.0,7ada1e42278c93ad863b88456bee6115283325ad90f415...,M,23.0,180.0,70.0,Algeria,ALG,2008 Summer,2008.0,Summer,Beijing,Rowing,Rowing Men's Lightweight Double Sculls,NaN,Algeria,NaN
314,3534.0,f2d8e64083d570c355631da5268ab700778cd7d60cd7d4...,M,23.0,177.0,70.0,Algeria,ALG,2000 Summer,2000.0,Summer,Sydney,Rowing,Rowing Men's Single Sculls,NaN,Algeria,NaN
424,13971.0,c1c7d113eba119f7e1eff20ef152ecd66e20cb4b2a5926...,M,26.0,178.0,73.0,Algeria,ALG,2016 Summer,2016.0,Summer,Rio de Janeiro,Rowing,Rowing Men's Single Sculls,NaN,Algeria,NaN
513,30029.0,68d0e60b4a44ab30cdd1eebe336f008a112209bcb3e88d...,M,27.0,182.0,70.0,Algeria,ALG,2008 Summer,2008.0,Summer,Beijing,Rowing,Rowing Men's Single Sculls,NaN,Algeria,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270931,48566.0,dc92b232c7e3d388850722caa60b698f39fc33a3ee746d...,F,20.0,170.0,67.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Rowing,Rowing Women's Single Sculls,NaN,Zimbabwe,NaN
271041,92718.0,b9ba8d293cf7147bd32305816a82c37898a7c1322867e7...,M,27.0,191.0,87.0,Zimbabwe,ZIM,2016 Summer,2016.0,Summer,Rio de Janeiro,Rowing,Rowing Men's Single Sculls,NaN,Zimbabwe,NaN
271062,114292.0,a03a800285bcb560ac42526f0d7698084b898a38d83e63...,F,35.0,NaN,NaN,Zimbabwe,ZIM,1992 Summer,1992.0,Summer,Barcelona,Rowing,Rowing Women's Coxless Pairs,NaN,Zimbabwe,NaN
271085,120082.0,95c1a7e653e0468c3335cdc759a3c773ace38b301b46d1...,F,25.0,175.0,72.0,Zimbabwe,ZIM,2012 Summer,2012.0,Summer,London,Rowing,Rowing Women's Single Sculls,NaN,Zimbabwe,NaN


In [75]:
rowing_rows[rowing_rows['ID']==120082.0]

,ID,Hash values,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
271085,120082.0,95c1a7e653e0468c3335cdc759a3c773ace38b301b46d1...,F,25.0,175.0,72.0,Zimbabwe,ZIM,2012 Summer,2012.0,Summer,London,Rowing,Rowing Women's Single Sculls,NaN,Zimbabwe,NaN
271086,120082.0,95c1a7e653e0468c3335cdc759a3c773ace38b301b46d1...,F,29.0,175.0,72.0,Zimbabwe,ZIM,2016 Summer,2016.0,Summer,Rio de Janeiro,Rowing,Rowing Women's Single Sculls,NaN,Zimbabwe,NaN


In [79]:
unique_rower_IDs = rowing_rows["ID"].unique()

years_rowing = []

for person in unique_rower_IDs:
    single_rower_years = rowing_rows[rowing_rows['ID']==person]["Year"].nunique()
    years_rowing.append(single_rower_years)

mean_years_rowing = st.mean(years_rowing)
mean_years_rowing

1.3299076362690256

In [81]:
max_value = max(years_rowing)
max_value


8

#### hur många olika evenemang finns i roddgrenen?

In [77]:
all_row_events=[rowing_rows["Event"].unique()]

rowing_rows[rowing_rows["Year"].isin(all_row_events)]

,ID,Hash values,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes


Alla 25 events har aldrig varit i ett och samma år. Google säger 14 roddevent senaste spelen.

#### FRÅGESTÄLLNING: Finns det någon korrelation mellan att få medalj i rodd och längd eller vikt?
Vi gör en correlation pearson plot där vi testar korrelationen mellan medaljer, längd och vikt.


In [78]:
corr_columns = rowing_rows[["Weight", "Height", "Medal"]]

# ofta saknas både weight och height för samma personer. Kanske ta bort de rader där en av dem saknas. 
# så att vi bara kollar korrelation för existerande höjd och vikt med medaljstatus. 
# här får varje person vara med flera gånger! 

# ingen medalj är 0, en medalj är 1 - eventuellt tydligare korrelation med medalj eller ej 
corr_columns["Medal"].fillna(0, inplace=True)
corr_columns["Medal"].mask(corr_columns["Medal"].isin(["Bronze", "Silver", "Gold"]) , other=1, inplace=True)

# innan vi tar bort rader med saknad height elr weight rader har vi 10595 rows
corr_columns.dropna(axis=0, how='any', subset=['Weight', 'Height'], inplace=True)
corr_columns
# kvar har vi 7794 rows 

# create a correlation table with pandas
corr_table = corr_columns.corr(method="pearson")

# set up the correlation plot

fig_corr= px.imshow(
    # pass the correlation matrix,
    corr_table,
    #show the correlation values in each cell,
    text_auto=True, 
    # set the colour scale
    color_continuous_scale="RdYlGn",
    zmin=-1,
    zmax=1, 
    title="Correlation betweeln weight, height and medal in rowing. 0 = no correlation"
 
)
fig_corr.show()

C:\Users\jubel\AppData\Local\Temp\ipykernel_5616\808835725.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\jubel\AppData\Local\Temp\ipykernel_5616\808835725.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jubel\AppData\Local\Temp\ipykernel_5616\808835725.py:9: SettingWithCopyWarning:


A value is trying to be set on a

### Svar på frågeställning: 

+1 Perfekt positiv korrelation – ökar den ena, ökar den andra exakt lika
0	Ingen linjär korrelation
–1	Perfekt negativ korrelation – ökar den ena, minskar den andra exakt lika

Mellan weight och height finns positiv korrelation (en ökar, andra ökar)

Men det verkar inte finnas nån korrelation mellan Medalj och Weight/Height. På nätet står det att tung och lång är bäst för rodd. Kanske att alla de som tävlar på elit-nivå har den ideala vikten och längden för prestera väl i rodd. Det verkar som en rimlig förklaring till att vi INTE ser en korrelation i denna datan. 
